# Create databases of RRSG/qMRSG Challenge data

## Imports

In [1]:
from pathlib import Path
import pandas as pd
import json
import nibabel as nib
import numpy as np

from src.database import *
from make_pooled_datasets import *

## Configurations

In [2]:
configFile = Path('configs/3T_human_T1maps.json')
data_folder_name = '3T_human_T1maps'
output_filename = data_folder_name + '_database' 
output_folder = Path('databases/')

## Download datasets

In [3]:
if not Path(data_folder_name).exists():
    make_pooled_dataset(configFile, data_folder_name)

## Create database

In [4]:
df = create_database(configFile, data_folder_name)

## Save database

In [5]:
df.to_excel(output_folder / (output_filename + '.xlsx')) # To excel
df.to_pickle(output_folder / (output_filename + '.pkl')) # To pickle (Python object)
df.to_csv(output_folder / (output_filename + '.csv')) # To csv

## Database manipulation demo

### View entire database

In [6]:
df

,OSF dataset,OSF link,NIFTI filename,Data type,contact,site name,MRI vendor,MRI version,MRI field,sample type,...,T1 - NIST sphere 5,T1 - NIST sphere 6,T1 - NIST sphere 7,T1 - NIST sphere 8,T1 - NIST sphere 9,T1 - NIST sphere 10,T1 - NIST sphere 11,T1 - NIST sphere 12,T1 - NIST sphere 13,T1 - NIST sphere 14
id,,,,,,,,,,,,,,,,,,,,,
1.001,refaatgabr_mcgovern_human,https://osf.io/2f8j5/download/,20200103_refaatgabr_mcgovern_human/20200103_re...,Magnitude,Refaat Gabr,UTHealth McGovern Medical School,Philips,Ingenia R5.4.1,3.0,None,...,None,None,None,None,None,None,None,None,None,None
2.001,mrel_usc_human,https://osf.io/z8dyw/download/,20200128_mrel_usc_human/20200128_mrel_usc_GE3T...,Magnitude,Nam Lee,Keck Medical Center of University of Southern ...,GE,Signa HDxt,3.0,None,...,None,None,None,None,None,None,None,None,None,None
2.002,mrel_usc_human,https://osf.io/z8dyw/download/,20200128_mrel_usc_human/20200128_mrel_usc_GE3T...,Magnitude,Nam Lee,Keck Medical Center of University of Southern ...,GE,Signa HDxt,3.0,None,...,None,None,None,None,None,None,None,None,None,None
2.003,mrel_usc_human,https://osf.io/z8dyw/download/,20200128_mrel_usc_human/20200128_mrel_usc_GE3T...,Magnitude,Nam Lee,Keck Medical Center of University of Southern ...,GE,Signa HDxt,3.0,None,...,None,None,None,None,None,None,None,None,None,None
2.004,mrel_usc_human,https://osf.io/z8dyw/download/,20200128_mrel_usc_human/20200206_mrel_usc_GE3T...,Magnitude,Nam Lee,Keck Medical Center of University of Southern ...,GE,Signa HDxt,3.0,None,...,None,None,None,None,None,None,None,None,None,None
2.005,mrel_usc_human,https://osf.io/z8dyw/download/,20200128_mrel_usc_human/20200228_mrel_usc_GE3T...,Magnitude,Nam Lee,Keck Medical Center of University of Southern ...,GE,Signa HDxt,3.0,None,...,None,None,None,None,None,None,None,None,None,None
2.006,mrel_usc_human,https://osf.io/z8dyw/download/,20200128_mrel_usc_human/20200228_mrel_usc_GE3T...,Magnitude,Nam Lee,Keck Medical Center of University of Southern ...,GE,Signa HDxt,3.0,None,...,None,None,None,None,None,None,None,None,None,None
3.001,iveslevesque_muhc_mgh_human,https://osf.io/n9edp/download/,20200203_iveslevesque_muhc_mgh_human/20200203_...,Magnitude,Ives Levesque,McGill University Health Centre - Montreal Gen...,Siemens,syngo MR E11,3.0,None,...,None,None,None,None,None,None,None,None,None,None
4.001,siyuanhu_casewestern_human,https://osf.io/jrzg6/download/,20200203_siyuanhu_casewestern_human/20200203_s...,Magnitude,Siyuan Hu,"Biomedical Engineering, Case Western Reserve U...",Siemens,Skyra,3.0,None,...,None,None,None,None,None,None,None,None,None,None


### View a column values for a specific database ID (i.e. row)

In [7]:
df.loc[1.001]

OSF dataset                                      refaatgabr_mcgovern_human
OSF link                                    https://osf.io/2f8j5/download/
NIFTI filename           20200103_refaatgabr_mcgovern_human/20200103_re...
Data type                                                        Magnitude
contact                                                        Refaat Gabr
site name                                 UTHealth McGovern Medical School
MRI vendor                                                         Philips
MRI version                                                 Ingenia R5.4.1
MRI field                                                                3
sample type                                                           None
phantom version                                                       None
phantom serial number                                                 None
phantom temperature                                                   None
age                      

### Get the value of a cell entry (row/column intersection)

In [8]:
df.loc[1.001]['T1 - genu (WM)']

array([1306.3, 1195.7, 1258.8, 1389.5, 1484.2, 1217.3, 1236.7, 1223.9,
       1314.8, 1347. , 1210.2, 1220.4, 1250.9, 1177.3, 1136.7, 1212.8,
       1193.6, 1268.3, 1199.9, 1099.1, 1225.8, 1282.4, 1344.7, 1279.1,
       1155.5])